In [43]:
import re
import numpy as np
from functools import cached_property

In [139]:
input_path_test = './day5_input_test.txt'
input_path_test_single = './day5_input_single_value.txt'
input_path = './day5_input.txt'

In [166]:
class Interval:
    def __init__(self, start = None, end = None, length = None):
        if start is None:
            start = end - length + 1
        if end is None:
            end = start + length - 1
        if length is None:
            length = end - start + 1
            
        self.start = start
        self.end = end
        self.length = length

    def __str__(self):
        return f"start: {self.start}, end: {self.end}, length: {self.length}"
        
class Map:
    def __init__(self, a, b, c):
        self.domain_interval = Interval(start=b, length = c)
        self.range_interval = Interval(start=a, length=c)
    
class IntervalMapper:
    def __init__(self, input_interval: Interval, reference_map: Map):
        self.input_interval = input_interval
        self.reference_map = reference_map
        
    def apply(self):
        domain_start = self.reference_map.domain_interval.start
        domain_end = self.reference_map.domain_interval.end
        
        input_start = self.input_interval.start
        input_end = self.input_interval.end
        
        range_start = self.reference_map.range_interval.start
        range_end = self.reference_map.range_interval.end
        
        if input_end < domain_start:
            return [(self.input_interval, False)]
        
        if input_start > domain_end:
            return [(self.input_interval, False)]
        
        if input_start >= domain_start and input_end <= domain_end:
            new_start = range_start + input_start - domain_start
            return [
                (
                    Interval(start=new_start, length = self.input_interval.length), 
                    True
                )
            ]
        
        else:
            if input_start < domain_start and input_end <= domain_end:
                interval_left = Interval(start=input_start, end=domain_start-1)
                interval_right = Interval(start=range_start, length=input_end - domain_start + 1)
                return [(interval_left, False), (interval_right, True)]
            
            if input_start >= domain_start and input_end > domain_end:
                interval_left = Interval(end = range_end, length = domain_end - input_start + 1)
                interval_right = Interval(start = domain_end + 1, end = input_end)
                return [(interval_left, True), (interval_right, False)]
            
            if input_start < domain_start and input_end > domain_end:
                interval_left = Interval(start=input_start, end = domain_start - 1)
                interval_center = Interval(start=range_start, end=range_end)
                interval_right = Interval(start=domain_end + 1, end=input_end)
                return [(interval_left, False), (interval_center, True), (interval_right, False)]

In [171]:
my_map = Map(50, 98, 2)
interval_in = Interval(start=79, length=14)
x = IntervalMapper(input_interval=interval_in, reference_map=my_map).apply()[0][0]
my_map2 = Map(52, 50, 48)
y = IntervalMapper(input_interval = x, reference_map = my_map2).apply()[0][0]
print(y)


start: 81, end: 94, length: 14


In [191]:
class PartB:
    def __init__(self, input_file):
        self.input_file = input_file

    @property
    def seeds(self):
        with open(self.input_file, 'r') as f:
            seed_pairs = f.readline().rstrip('\n')
            
        seed_pairs = seed_pairs.split()[1:]
        seed_pairs = [int(seed) for seed in seed_pairs]
        
        seed_values = []
        while len(seed_pairs) > 0:
            start_seed = seed_pairs[0]
            num_seeds = seed_pairs[1]
            seed_values.append((Interval(start = start_seed, length = num_seeds), False))
            seed_pairs = seed_pairs[2:]
            
        return seed_values
    
    @cached_property
    def name_mappings_dict(self):
        with open(self.input_file, 'r') as f:
            all_lines = f.readlines()
        all_lines = [line.rstrip('\n') for line in all_lines]
        all_lines = [line for line in all_lines if line != '']
        all_lines = all_lines[1:]
        
        name_map_dict = {}
        current_name = None
        ii = 0
        
        for line in all_lines:
            if line.endswith(':'):
                if current_name is not None:
                    name_map_dict[ii] = (current_name, current_map_list)
                current_name = line.split()[0]
                current_map_list = []
                ii += 1
            else:
                vals = line.split()
                a, b, c = vals
                current_map_list.append({"a": int(a), "b": int(b), "c": int(c)})
        
        name_map_dict[ii] = (current_name, current_map_list)
        return name_map_dict
    
    def get_all_intervals(self):
        mapping_keys = list(self.name_mappings_dict.keys())
        mapping_keys.sort()
        
        current_inputs = self.seeds
        for mapping_key in mapping_keys:
            name, map_list = self.name_mappings_dict[mapping_key]
            print(f"Mapping {name}")
            for map_dict in map_list:
                temp_output = []
                reference_map = Map(**map_dict)
                for interval, mapped_yet in current_inputs:
                    if not mapped_yet:
                        mapper = IntervalMapper(interval, reference_map)
                        temp_output += mapper.apply()
                    else:
                        temp_output += [(interval, mapped_yet)]
                    
                current_inputs = temp_output
            current_inputs = [(interval, False) for interval, _ in current_inputs]
                
            # for interval, mapped_yet in current_inputs: 
            #     print(interval, mapped_yet)
                
            
        return current_inputs
    


In [195]:
part_b_test = PartB(input_path)
part_b_test.seeds

[(<__main__.Interval at 0x7fef92127eb0>, False),
 (<__main__.Interval at 0x7fef607b3250>, False),
 (<__main__.Interval at 0x7fef607b3820>, False),
 (<__main__.Interval at 0x7fefa0290490>, False),
 (<__main__.Interval at 0x7fefa0290af0>, False),
 (<__main__.Interval at 0x7fefa02900d0>, False),
 (<__main__.Interval at 0x7fefa0290370>, False),
 (<__main__.Interval at 0x7fefa0290040>, False),
 (<__main__.Interval at 0x7fefa0290eb0>, False),
 (<__main__.Interval at 0x7fefa0290670>, False)]

In [196]:
intervals = part_b_test.get_all_intervals()

Mapping seed-to-soil
Mapping soil-to-fertilizer
Mapping fertilizer-to-water
Mapping water-to-light
Mapping light-to-temperature
Mapping temperature-to-humidity
Mapping humidity-to-location


In [197]:
min([interval[0].start for interval in intervals])

10834440